In [ ]:
# Download data scraped from arxiv.org

In [ ]:
# 1) pip install bs4, if needed

In [ ]:
import re 
from bs4 import BeautifulSoup, NavigableString, Tag
import pandas as pd

In [ ]:
# Configurations
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0"}
page = 'https://arxiv.org/help/api/user-manual'

In [ ]:
def find_table_after_heading(soup, heading_text):
    # Stolen from: https://stackoverflow.com/a/51936486
    for header in soup.find_all('h3', text=re.compile(heading_text)):
        nextNode = header
        while True:
            nextNode = nextNode.nextSibling
            if nextNode is None:
                break
            if isinstance(nextNode, Tag):
                if nextNode.name == "h3":
                    break
                #print(nextNode)
                return nextNode
    return None

In [ ]:
with requests.Session() as s:
    # Get data
    r = s.get(page, headers = headers)
    soup = BeautifulSoup(r.content, 'lxml')
    # Find table data we are looking for
    table_node = find_table_after_heading(soup, '5.3. Subject Classifications')

In [ ]:
#print(table_node)

In [ ]:
# Iterate over table data and convert dataframe
data = []
cols = ['tag','name']
for tr in table_node.find_all('tr')[2:]:
    tds = tr.find_all('td')
    data.append([tds[0].text, tds[1].text])
    #print(f"tag: {tds[0].text}, name: {tds[1].text}")
df = pd.DataFrame.from_records(data, columns=cols)

In [ ]:
# Save dataframe to disk
df.to_csv('../data/subject-classifications.csv', index=False)

In [ ]:
# Download dataset from Kaggle

In [ ]:
# 1) Go to your Kaggle user account: https://www.kaggle.com/<user>/account
# 2) Click [Create New API Token] and download kaggle.json file to your ~/.kaggle/ folder (create if needed)
# 3) pip install kaggle, if needed

In [38]:
# From: https://stackoverflow.com/questions/55934733/documentation-for-kaggle-api-within-python
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

In [39]:
# Download all files of a dataset
# Signature: dataset_download_files(dataset, path=None, force=False, quiet=True, unzip=False)
# Url: https://www.kaggle.com/Cornell-University/arxiv
api.dataset_download_files('Cornell-University/arxiv', path='../data/')